In [1]:
!pip install pmdarima

In [2]:
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima
from sklearn.preprocessing import MinMaxScaler

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

In [4]:
train = pd.read_csv('/home/lab10/Dacon/contest/public_data/train.csv')
train.head(5)

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),...,청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,...,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0
2,2016-01-03,일요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-01-04,월요일,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,...,38525.5,7631.0,500702.0,2046.0,620539.0,2018.0,2703.8,3885.0,0.0,0.0
4,2016-01-05,화요일,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,...,32615.0,6926.0,147638.0,2268.0,231958.0,2178.0,8810.0,2853.0,0.0,0.0


In [5]:
# 시간 데이터 인덱스
train['date'] = pd.to_datetime(train['date'])
train = train.set_index(train['date'])
train = train.iloc[:, 1:]
train.head(5)

,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),...,청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,15019.0,...,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0
2016-01-03,일요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-04,월요일,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,141638.0,...,38525.5,7631.0,500702.0,2046.0,620539.0,2018.0,2703.8,3885.0,0.0,0.0
2016-01-05,화요일,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,126207.8,...,32615.0,6926.0,147638.0,2268.0,231958.0,2178.0,8810.0,2853.0,0.0,0.0


In [6]:
train_1=train.drop(['요일'], axis=1)
train_2=train_1.diff()
train_2=train_2.dropna()
ss_train2=MinMaxScaler().fit_transform(train_2)
ss_train2=pd.DataFrame(ss_train2)
ss_train2.columns = ['배추_거래량(kg)', '배추_가격(원/kg)', '무_거래량(kg)', '무_가격(원/kg)',
       '양파_거래량(kg)', '양파_가격(원/kg)', '건고추_거래량(kg)', '건고추_가격(원/kg)',
       '마늘_거래량(kg)', '마늘_가격(원/kg)', '대파_거래량(kg)', '대파_가격(원/kg)',
       '얼갈이배추_거래량(kg)', '얼갈이배추_가격(원/kg)', '양배추_거래량(kg)', '양배추_가격(원/kg)',
       '깻잎_거래량(kg)', '깻잎_가격(원/kg)', '시금치_거래량(kg)', '시금치_가격(원/kg)',
       '미나리_거래량(kg)', '미나리_가격(원/kg)', '당근_거래량(kg)', '당근_가격(원/kg)',
       '파프리카_거래량(kg)', '파프리카_가격(원/kg)', '새송이_거래량(kg)', '새송이_가격(원/kg)',
       '팽이버섯_거래량(kg)', '팽이버섯_가격(원/kg)', '토마토_거래량(kg)', '토마토_가격(원/kg)',
       '청상추_거래량(kg)', '청상추_가격(원/kg)', '백다다기_거래량(kg)', '백다다기_가격(원/kg)',
       '애호박_거래량(kg)', '애호박_가격(원/kg)', '캠벨얼리_거래량(kg)', '캠벨얼리_가격(원/kg)',
       '샤인마스캇_거래량(kg)', '샤인마스캇_가격(원/kg)']
ss_train2

,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),...,청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,0.466885,0.501601,0.460068,0.660821,0.464140,0.792016,0.499703,0.541366,0.471947,0.925046,...,0.466201,0.813080,0.391000,0.709522,0.405487,0.778772,0.440422,0.501982,0.448995,0.500000
1,0.450077,0.431391,0.437133,0.373854,0.437295,0.050911,0.499696,0.473082,0.457222,0.051910,...,0.396842,0.215365,0.390760,0.235908,0.377456,0.231876,0.439760,0.414711,0.448995,0.500000
2,0.606352,0.517499,0.691411,0.669589,0.703791,0.778710,0.500547,0.521080,0.534019,0.903915,...,0.692213,0.761173,0.529373,0.702448,0.845419,0.733915,0.441107,0.542520,0.448995,0.500000
3,0.431925,0.462655,0.409147,0.533280,0.426437,0.415100,0.500201,0.506846,0.457020,0.502591,...,0.391527,0.491408,0.293224,0.497642,0.107210,0.523448,0.442385,0.435987,0.448995,0.500000
4,0.445829,0.466496,0.517300,0.512156,0.425314,0.435927,0.500378,0.515601,0.447585,0.458656,...,0.421144,0.468431,0.402601,0.502695,0.419223,0.542139,0.439594,0.442444,0.448995,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1727,0.442838,0.469910,0.441304,0.531287,0.451181,0.423489,0.499792,0.493232,0.477920,0.444861,...,0.416767,0.516391,0.387430,0.461374,0.369567,0.507589,0.450633,0.455942,0.489764,0.498225
1728,0.460885,0.461161,0.423403,0.525707,0.428099,0.411339,0.498571,0.519621,0.460940,0.509529,...,0.455828,0.503511,0.399179,0.472490,0.427757,0.497508,0.430864,0.458304,0.479439,0.498800
1729,0.435719,0.463402,0.424332,0.542846,0.452310,0.423199,0.498574,0.505408,0.456463,0.530420,...,0.478151,0.501019,0.395079,0.480126,0.411677,0.475306,0.455799,0.459929,0.463070,0.497400
1730,0.288476,0.605847,0.209860,0.542846,0.224343,0.448944,0.498610,0.506675,0.410788,0.461526,...,0.018285,0.524125,0.300402,0.544801,0.107710,0.523448,0.252329,0.455660,0.000000,0.500788


In [7]:
# plot_acf(train_2['배추_가격(원/kg)'])
# plot_pacf(train_2['배추_가격(원/kg)'])

In [8]:
# arima model 중 AR
for col in ss_train2:
    print('Ex')
    p=range(0,2)
    d=range(1,2)
    q=range(0,2)
    pdq=list(itertools.product(p, d, q))

    aic=[]
    for i in pdq:
        model = ARIMA(ss_train2[col], order=(i))
        model_fit=model.fit()
        print(f'Arima: {i} >> AIC : {round(model_fit.aic,2)}')
        aic.append(round(model_fit.aic,2))


Ex
Arima: (0, 1, 0) >> AIC : -1593.46
Arima: (0, 1, 1) >> AIC : -3411.56


/home/ubuntu/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


Arima: (1, 1, 0) >> AIC : -2455.41


/home/ubuntu/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


Arima: (1, 1, 1) >> AIC : -3781.58
Ex
Arima: (0, 1, 0) >> AIC : -3723.6
Arima: (0, 1, 1) >> AIC : -5615.12
Arima: (1, 1, 0) >> AIC : -4719.33


ValueError: The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.